<a href="https://colab.research.google.com/github/tbrar/CPSC-501-A4-TensorFlow/blob/master/CHDModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

TensorFlow 2.x selected.


In [4]:
from google.colab import files
uploaded = files.upload()

Saving heart_train.csv to heart_train.csv


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
import functools

import numpy as np
import tensorflow as tf

In [0]:
train_file_path = "heart_train.csv"
test_file_path = "heart_test.csv"

In [83]:
LABEL_COLUMN = 'chd'
LABELS = [0, 1]

def get_dataset(file_path, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=5, # Artificially small to make examples easier to show.
      label_name=LABEL_COLUMN,
      na_value="?",
      num_epochs=1,
      ignore_errors=True, 
      **kwargs)
  return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

show_batch(raw_train_data)




row.names           : [436 329 278 162 386]
sbp                 : [132 128 174 208 126]
tobacco             : [ 0.    0.    2.02 27.4  19.6 ]
ldl                 : [3.55 2.98 6.57 3.12 6.03]
adiposity           : [ 8.66 12.59 31.9  26.63 34.99]
famhist             : [b'Present' b'Absent' b'Present' b'Absent' b'Absent']
typea               : [61 65 50 66 49]
obesity             : [18.5  20.74 28.75 27.45 26.99]
alcohol             : [ 3.87  2.06 11.83 33.07 55.89]
age                 : [16 19 64 62 44]


In [84]:
SELECT_COLUMNS = ['sbp', 'tobacco', 'ldl', 'adiposity', 'typea', 'obesity', 'alcohol', 'age', 'chd']
DEFAULTS = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0]
temp_dataset = get_dataset(train_file_path, 
                           select_columns=SELECT_COLUMNS,
                           column_defaults = DEFAULTS)

example_batch, labels_batch = next(iter(temp_dataset)) 
def pack(features, label):
  return tf.stack(list(features.values()), axis=-1), label

packed_dataset = temp_dataset.map(pack)

for features, labels in packed_dataset.take(1):
  print(features.numpy())
  print()
  print(labels.numpy())

show_batch(raw_train_data)

example_batch, labels_batch = next(iter(temp_dataset)) 


[[136.     3.15   4.37  20.22  59.    25.12  47.16  31.  ]
 [144.     2.4    8.13  35.61  46.    27.38  13.37  60.  ]
 [118.     0.     2.39  12.13  49.    18.46   0.26  17.  ]
 [152.     1.68   3.58  25.43  50.    27.03   0.    32.  ]
 [136.     5.     4.19  23.99  68.    27.8   25.86  35.  ]]

[1 0 1 0 0]
row.names           : [400 106 315 215 312]
sbp                 : [126 158 174 134 174]
tobacco             : [8.75 3.6  0.   0.   3.5 ]
ldl                 : [6.06 2.97 3.27 2.4  5.26]
adiposity           : [32.72 30.11 35.4  21.11 21.97]
famhist             : [b'Present' b'Absent' b'Absent' b'Absent' b'Present']
typea               : [33 63 58 57 36]
obesity             : [27.   26.64 37.71 22.45 22.04]
alcohol             : [ 62.43 108.    24.95   1.37   8.33]
age                 : [55 64 44 18 59]


In [0]:
class PackNumericFeatures(object):
  def __init__(self, names):
    self.names = names

  def __call__(self, features, labels):
    numeric_features = [features.pop(name) for name in self.names]
    numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
    numeric_features = tf.stack(numeric_features, axis=-1)
    features['numeric'] = numeric_features

    return features, labels

In [86]:
NUMERIC_FEATURES = ['sbp', 'tobacco', 'ldl', 'adiposity', 'typea', 'obesity', 'alcohol', 'age']

packed_train_data = raw_train_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

packed_test_data = raw_test_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

show_batch(packed_train_data)

example_batch, labels_batch = next(iter(packed_train_data)) 

row.names           : [280 289 304 346 328]
famhist             : [b'Absent' b'Absent' b'Absent' b'Absent' b'Present']
numeric             : [[1.140e+02 0.000e+00 2.630e+00 9.690e+00 4.500e+01 1.789e+01 0.000e+00
  1.600e+01]
 [1.340e+02 2.000e-02 2.800e+00 1.884e+01 4.500e+01 2.482e+01 0.000e+00
  1.700e+01]
 [1.700e+02 0.000e+00 3.120e+00 3.715e+01 4.700e+01 3.542e+01 0.000e+00
  5.300e+01]
 [1.370e+02 1.200e+00 3.140e+00 2.387e+01 6.600e+01 2.413e+01 4.500e+01
  3.700e+01]
 [1.320e+02 0.000e+00 6.630e+00 2.958e+01 3.700e+01 2.941e+01 2.570e+00
  6.200e+01]]


In [87]:
import pandas as pd
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc

,sbp,tobacco,ldl,adiposity,typea,obesity,alcohol,age
count,396.000000,396.000000,396.000000,396.000000,396.000000,396.000000,396.000000,396.000000
mean,139.088384,3.534470,4.721641,25.417854,52.611111,25.980404,17.155328,42.727273
std,20.883153,4.622259,1.969109,7.862184,9.908200,4.185559,24.696052,14.758524
min,101.000000,0.000000,0.980000,6.740000,13.000000,14.700000,0.000000,15.000000
25%,125.500000,0.050000,3.300000,19.825000,46.000000,22.757500,0.510000,31.000000
50%,134.000000,1.800000,4.325000,26.115000,53.000000,25.780000,7.780000,44.500000
75%,148.500000,5.152500,5.632500,31.102500,59.000000,28.402500,23.737500,55.250000
max,218.000000,31.200000,14.160000,42.490000,77.000000,45.720000,147.190000,64.000000


In [88]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

def normalize_numeric_data(data, mean, std):
  # Center the data
  return (data-mean)/std

# See what you just created.
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()


array([[-0.14788853, -0.4704344 , -0.79307026, -1.3288742 ,  0.34202874,
        -0.23901333, -0.39906493, -1.2689123 ],
       [ 1.0013632 ,  0.14398377,  1.0351683 ,  1.5990655 ,  0.84666127,
         1.6555964 , -0.5695375 ,  0.76381135],
       [-0.9140563 , -0.76466286, -0.5848543 , -0.27954763,  0.5438818 ,
         0.29138207, -0.69465876, -0.72685266],
       [ 0.1394244 , -0.76466286, -0.26999095, -0.93839747,  0.34202874,
        -0.55677253,  0.14677131, -0.0492781 ],
       [ 2.6294696 , -0.39687732,  0.81171685,  1.0495994 , -0.56630975,
         0.9985754 , -0.68696517,  0.89932626]], dtype=float32)

In [90]:
CATEGORIES = {
    'famhist': ['Absent', 'Present'],
}

categorical_columns = []
for feature, vocab in CATEGORIES.items():
  cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
  categorical_columns.append(tf.feature_column.indicator_column(cat_col))

# See what you just created.
categorical_columns

categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy()[0])


Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
[0. 1.]


In [92]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)
print(preprocessing_layer(example_batch).numpy()[0])


[ 0.          1.         -0.14788853 -0.4704344  -0.79307026 -1.3288742
  0.34202874 -0.23901333 -0.39906493 -1.2689123 ]


In [96]:
model = tf.keras.Sequential([
  preprocessing_layer,
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid'),
])

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

train_data = packed_train_data.shuffle(500)
test_data = packed_test_data

model.fit(train_data, epochs=20)

test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

predictions = model.predict(test_data)

# Show some results
for prediction, chd in zip(predictions[:10], list(test_data)[0][1][:10]):
  print("Predicted chd: {:.2%}".format(prediction[0]),
        " | Actual outcome: ",
        ("Has CHD" if bool(chd) else "Doesn't have CHD"))


Epoch 1/20
80/80 [==============================] - 1s 12ms/step - loss: 0.5700 - accuracy: 0.6944
Epoch 2/20
80/80 [==============================] - 0s 3ms/step - loss: 0.5088 - accuracy: 0.7525
Epoch 3/20
80/80 [==============================] - 0s 3ms/step - loss: 0.5199 - accuracy: 0.7424
Epoch 4/20
80/80 [==============================] - 0s 3ms/step - loss: 0.4899 - accuracy: 0.7727
Epoch 5/20
80/80 [==============================] - 0s 3ms/step - loss: 0.4726 - accuracy: 0.7727
Epoch 6/20
80/80 [==============================] - 0s 3ms/step - loss: 0.4691 - accuracy: 0.7803
Epoch 7/20
80/80 [==============================] - 0s 3ms/step - loss: 0.4606 - accuracy: 0.7727
Epoch 8/20
80/80 [==============================] - 0s 3ms/step - loss: 0.4484 - accuracy: 0.7652
Epoch 9/20
80/80 [==============================] - 0s 3ms/step - loss: 0.4337 - accuracy: 0.7879
Epoch 10/20
80/80 [==============================] - 0s 3ms/step - loss: 0.4310 - accuracy: 0.8030
Epoch 11/20
80/80 